<b> Análisis previo de los datos </b>

In [1]:
import pandas as pd
import numpy as np
import math

trainingStations = 'trainingStations.csv'
deployStations = 'deployStations.csv'
testStations = 'testStations.csv'
Stations_DistanceMatrix = 'Stations_DistanceMatrix.csv'

df_trainingStations = pd.read_csv(trainingStations)
df_deployStations = pd.read_csv(deployStations)
df_testStations = pd.read_csv(testStations)
df_Stations_DistanceMatrix = pd.read_csv(Stations_DistanceMatrix)



In [2]:
#Sustituimos los . de las colmnas por _
df_trainingStations.rename(columns={'windMaxSpeed.m.s':'windMaxSpeed_m_s',
                          'windMeanSpeed.m.s':'windMeanSpeed_m_s',
                          'windDirection.grades':'windDirection_grades',
                          'temperature.C':'temperature_C',
                          'relHumidity.HR':'relHumidity_HR',
                          'airPressure.mb':'airPressure_mb',
                          'precipitation.l.m2':'precipitation_l_m2'}, 
                 inplace=True)

df_testStations.rename(columns={'windMaxSpeed.m.s':'windMaxSpeed_m_s',
                          'windMeanSpeed.m.s':'windMeanSpeed_m_s',
                          'windDirection.grades':'windDirection_grades',
                          'temperature.C':'temperature_C',
                          'relHumidity.HR':'relHumidity_HR',
                          'airPressure.mb':'airPressure_mb',
                          'precipitation.l.m2':'precipitation_l_m2'}, 
                 inplace=True)

In [3]:
#Dataframe en el que se calcula la media para las columnas que queremos sustituir los datos
#faltantes. En el se calcula la media y se agrupa por año, mes y día.

df_test_mean_day = df_testStations[['station','year','month','day',\
                                            'windMaxSpeed_m_s','windMeanSpeed_m_s',\
                                            'windDirection_grades','temperature_C',\
                                            'relHumidity_HR','airPressure_mb',\
                                            'precipitation_l_m2','bikes_3h_ago']]\
                                            .groupby(['station','year','month','day']).mean()

In [4]:
#Bucle en el que se rellenan los datos faltantes. En cada uno de los for se localizan los valores
#faltantes y estos se sustituyen por los que hemos calculado en el dataframe de la celda anterior.
import math

print('Reemplazando valores faltantes para bikes_3h_ago...')

counter = 0
for index, row in df_testStations.iterrows():
    if math.isnan(row['bikes_3h_ago']) == True:
        mean = df_test_mean_day[df_test_mean_day.index.get_level_values('station') == row['station']]
        mean = mean[mean.index.get_level_values('year') == row['year']]
        mean = mean[mean.index.get_level_values('month') == row['month']]
        mean = mean[mean.index.get_level_values('day') == row['day']]
        for index, row in mean.iterrows():
            mean_result = row['bikes_3h_ago']
        df_testStations.loc[counter,'bikes_3h_ago'] = mean_result
    counter = counter+1
    
print('Valores para bikes_3h_ago reemplazados') 
print('----------------------------------------')
print('Se han sustituido los valores faltantes por la media.')

Reemplazando valores faltantes para bikes_3h_ago...
Valores para bikes_3h_ago reemplazados
----------------------------------------
Se han sustituido los valores faltantes por la media.


In [5]:
#Eliminamos la columna year, ya que en los datos de test solo evaluamos 1 año.
del df_testStations["year"]

In [6]:
#Guardamos el csv en un archivo con la media en los valores faltantes.
df_testStations.to_csv('testStationsMissing.csv', index=False)

In [7]:
testStations_mean = 'testStationsMissing.csv'
df_testStations_mean = pd.read_csv(testStations_mean)

In [8]:
#Reemplazamos los valores faltantes por la media total de los demás (es probable que 
#durante todo un día no se hayan recogido datos, es por eso que algunas columnas siguen 
#con datos faltantes)
counter = 0
mean_whole_col = df_testStations_mean[['bikes_3h_ago']].mean(skipna = True)
for index, row in df_testStations_mean.iterrows():
    if math.isnan(row['bikes_3h_ago']) == True:
        df_testStations_mean.loc[counter,'bikes_3h_ago'] = mean_whole_col.values[0]
    counter = counter + 1

In [9]:
#Primero pasamos las variables cualitativas a cuantitativas (solo los días de la semana):
# Monday -> 1
# Tuesday -> 2
# Wednesday -> 3
# Thursday -> 4
# Friday -> 5
# Saturday -> 6
# Sunday -> 7

def switch_weekday(arg):
    switcher = {
        "Monday": 1,
        "Tuesday": 2,
        "Wednesday": 3,
        "Thursday": 4,
        "Friday": 5,
        "Saturday": 6,
        "Sunday": 7,
    }
    return switcher.get(arg, "Invalid Weekday")

In [10]:
col_to_change = ["weekday"]

print("Cambiando variable cualitativa por cuantitativa...")

for index, row in df_testStations_mean.iterrows():
    df_testStations_mean.loc[index,col_to_change] = switch_weekday(row["weekday"])
print("Variables cambiadas satisfactoriamente.")

Cambiando variable cualitativa por cuantitativa...
Variables cambiadas satisfactoriamente.


In [11]:
df_testStations_mean.to_csv('testStationsMissing.csv', index=False)

In [12]:
#Normalizamos todas las columnas menos Year, en test solo evaluamos sobre 2014
df_testStations_mean_mmn = df_testStations_mean

cols_to_norm = ["station", "latitude", "longitude", "numDocks", "timestamp", "month", \
                "day", "hour", "weekday", "weekhour", "windMaxSpeed_m_s", \
                "windMeanSpeed_m_s", "windDirection_grades", "temperature_C", "relHumidity_HR", \
                "airPressure_mb", "precipitation_l_m2", "bikes_3h_ago", \
                "full_profile_3h_diff_bikes", "full_profile_bikes", \
                "short_profile_3h_diff_bikes", "short_profile_bikes"]
df_testStations_mean_mmn[cols_to_norm] = df_testStations_mean_mmn[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))


In [13]:
df_testStations_mean_zscore = df_testStations_mean
#Valor absoluto de la Z-Score Normalization.
df_testStations_mean_zscore[cols_to_norm] = df_testStations_mean_zscore[cols_to_norm].apply(lambda x: (x - x.mean()) / (x.std(ddof=0)))

In [14]:
df_testStations_mean_mmn.to_csv('testStationsMissingNormMMN.csv', index=False)
df_testStations_mean_zscore.to_csv('trainingStationsMissingNormZScore.csv', index=False)